# A Stock Investing Inception

### [The Safe Bet Index: S&P500](#sp500)

1. [History](#sp500_history)
1. [Yearly Gain Loss](#sp500_yearly_gain_loss)
1. [Expected Return](#sp500_expected_return)
1. [Monthly Return](#sp500_monthly_return)
1. [Some huge event cause drastic drop](#sp500_event)
1. [How often new all time high](#sp500_new_high)
1. [The shallow pattern of most profitable month](#sp500_month_bias)
1. [When to join?](#sp500_good_timing)

### Fundamental

1. [Getting data](#fundamental_stuff)
1. [Calculate Intrinsic Value](#intrinsic_value)
1. [Undervalue big companies](#undervalue_companies)

[Dependencies](#dependencies)

In [ ]:
import copy
import datetime
import json
import re
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import requests
import seaborn as sns
import tqdm
import yfinance as yf
from bs4 import BeautifulSoup
from IPython.display import (
    Javascript,
    set_matplotlib_formats,
)
from matplotlib.lines import Line2D
from scipy import signal
from sklearn import preprocessing

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(
    color=['0C5DA5', '00B945', 'FF9500', 'FF2C00', '845B97', '474747', '9e9e9e']
)

In [ ]:
custom_style = {
    'xtick.direction': 'in',
    'xtick.major.size': 3,
    'xtick.major.width': 0.5,
    'xtick.minor.size': 1.5,
    'xtick.minor.width': 0.5,
    'xtick.minor.visible': True,
    'xtick.top': True,
    'xtick.labelsize': 14,
    'ytick.direction': 'in',
    'ytick.major.size': 3,
    'ytick.major.width': 0.5,
    'ytick.minor.size': 1.5,
    'ytick.minor.width': 0.5,
    'ytick.minor.visible': True,
    'ytick.right': True,
    'ytick.labelsize': 14,
    'axes.linewidth': 0.5,
    'axes.labelsize': 16,
    'grid.linewidth': 0.5,
    'lines.linewidth': 1.0,
    'legend.frameon': False,
    'legend.fontsize': 14,
    'font.family': 'serif',
    'font.size': 20,
    'mathtext.fontset': 'dejavuserif',
    'text.usetex': True,
}

plt.style.use(custom_style)
set_matplotlib_formats('svg')

## The Safe Bet Index: S&P500<a id='sp500'></a>

The Standard and Poor's 500, or simply the S&P 500, is a stock market index tracking the performance of 500 large companies listed on stock exchanges in the United States.

In [ ]:
%%time
sp500 = yf.Ticker('^GSPC')
sp500 = sp500.history(period='max')

## S&P500 History<a id='sp500_history'></a>

In [ ]:
signal_detrended = signal.detrend(sp500.Close.values)
fig, ax1 = plt.subplots(figsize=(8, 4))
color_ax1 = 'tab:blue'
ax1.plot(sp500.index, sp500['Close'], color=color_ax1, label='Closing price')
ax1.plot(sp500.index, signal_detrended, color='r', label='Detrended')
ax1.set_ylabel('Closing Value', color=color_ax1)
ax1.tick_params(axis='y', labelcolor=color_ax1)
ax1.grid(axis='y')
ax1.legend()
ax2 = ax1.twinx()
color_ax2 = 'black'
ax2.plot(sp500.index, sp500['Volume'], color=color_ax2, alpha=0.2)
ax2.set_ylabel('Volume Value', color=color_ax2)
ax2.tick_params(axis='y', labelcolor=color_ax2)
plt.title('SP500 History Closing')
fig.tight_layout()
plt.show()

## S&P500 Yearly Gain & Loss<a id='sp500_yearly_gain_loss'></a>

In [ ]:
sp500['Year'] = pd.DatetimeIndex(sp500.reset_index()['Date']).year
YEAR_LIST = sp500['Year'].unique().tolist()
first_day_of_years = sp500.groupby('Year').first()['Close'].values
last_day_of_years = sp500.groupby('Year').last()['Close'].values
percent_change_every_years = (last_day_of_years / first_day_of_years * 100 - 100) * 100
color_list = ['green' if v > 0 else 'red' for v in percent_change_every_years]
sp500.drop(columns='Year', inplace=True)

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(YEAR_LIST, percent_change_every_years, color=color_list)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_title('Yearly Gain or Loss')
ax.set_xlabel('Year')
ax.set_ylabel('Percent Change')
ax.grid(axis='y')
plt.show()

postive_negative_count = [
    'Positive Growth Year' if v > 0 else 'Negative Growth Year'
    for v in percent_change_every_years
]
postive_negative_count = Counter(postive_negative_count)
index_year_loss_30 = [
    idx for idx, v in enumerate(percent_change_every_years) if v < -30
]
year_loss_30 = [YEAR_LIST[idx] for idx in index_year_loss_30]
index_year_loss_20 = [
    idx for idx, v in enumerate(percent_change_every_years) if v < -20
]
year_loss_20 = [YEAR_LIST[idx] for idx in index_year_loss_20]
year_loss_20 = [v for v in year_loss_20 if v not in year_loss_30]
index_year_loss_10 = [
    idx for idx, v in enumerate(percent_change_every_years) if v < -10
]
year_loss_10 = [YEAR_LIST[idx] for idx in index_year_loss_10]
year_loss_10 = [v for v in year_loss_10 if v not in year_loss_20]
index_year_gain_30 = [idx for idx, v in enumerate(percent_change_every_years) if v > 30]
year_gain_30 = [YEAR_LIST[idx] for idx in index_year_gain_30]
index_year_gain_20 = [idx for idx, v in enumerate(percent_change_every_years) if v > 20]
year_gain_20 = [YEAR_LIST[idx] for idx in index_year_gain_20]
year_gain_20 = [v for v in year_gain_20 if v not in year_gain_30]
index_year_gain_10 = [idx for idx, v in enumerate(percent_change_every_years) if v > 10]
year_gain_10 = [YEAR_LIST[idx] for idx in index_year_gain_10]
year_gain_10 = [v for v in year_gain_10 if v not in year_gain_20]

print('Positive Growth Year Count: ', postive_negative_count['Positive Growth Year'])
print('Negative Growth Year Count: ', postive_negative_count['Negative Growth Year'])
print(
    '\nYear Surged >= 30%: {} \nYear: {}\n'.format(
        sum(v < -30 for v in percent_change_every_years), year_loss_30
    )
)
print(
    '\nYear Surged >= 20%: {} \nYear: {}\n'.format(
        sum(v < -20 for v in percent_change_every_years), year_loss_20
    )
)
print(
    '\nYear Surged >= 10%: {} \nYear: {}\n'.format(
        sum(v < -10 for v in percent_change_every_years), year_loss_10
    )
)
print(
    '\nYear Soared >= 30%: {} \nYear: {}\n'.format(
        sum(v > 30 for v in percent_change_every_years), year_gain_30
    )
)
print(
    '\nYear Soared >= 20%: {} \nYear: {}\n'.format(
        sum(v > 20 for v in percent_change_every_years), year_gain_20
    )
)
print(
    '\nYear Soared >= 10%: {} \nYear: {}\n'.format(
        sum(v > 10 for v in percent_change_every_years), year_gain_10
    )
)

## S&P500 Expected Return<a id='sp500_expected_return'></a>

In [ ]:
log_returns = np.log(sp500.Close).diff().dropna()
log_returns = log_returns * 100

fig, ax = plt.subplots(figsize=(10, 5))
log_returns.plot(
    ax=ax, title=r'S\&P500 Daily Log Returns', xlabel='Date', ylabel='Percentage %'
)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.grid(axis='y')
plt.show()

## S&P500 Monthly Return<a id='sp500_monthly_return'></a>

In [ ]:
monthly_sp500 = sp500[['Close']].resample('M').last()
monthly_sp500_return = pd.DataFrame()
outliers_threshold = 0.01
lags = [1, 2, 3, 6, 9, 12]
for lag in lags:
    monthly_sp500_return[f'return_{lag}m'] = (
        monthly_sp500.pct_change(lag)
        .stack()
        .pipe(
            lambda x: x.clip(
                lower=x.quantile(outliers_threshold),
                upper=x.quantile(1 - outliers_threshold),
            )
        )
        .add(1)
        .pow(1 / lag)
        .sub(1)
    )

monthly_sp500_return = monthly_sp500_return.swaplevel().dropna()
sns.clustermap(
    monthly_sp500_return.corr('spearman'), annot=True, center=0, cmap='Blues'
);

## S&P500: Event causes huge crash after 2000s<a id='sp500_event'></a>

In [ ]:
sp500_event = sp500.loc['1995-01-01':]

fig, ax = plt.subplots(figsize=(10, 5))
ax.axvspan(
    datetime.date(1998, 7, 1),
    datetime.date(1998, 10, 31),
    color='#CCBB44',
    alpha=0.2,
    label='Asia Economic Crisis',
)
ax.axvspan(
    datetime.date(2000, 3, 1),
    datetime.date(2002, 10, 31),
    color='#4477AA',
    alpha=0.2,
    label='Dot-com buble',
)
ax.axvspan(
    datetime.date(2007, 10, 1),
    datetime.date(2009, 1, 1),
    color='#EE6677',
    alpha=0.2,
    label='Great Recession',
)
ax.axvspan(
    datetime.date(2018, 1, 1),
    datetime.date(2019, 6, 1),
    color='#228833',
    alpha=0.2,
    label='US China trade war',
)
ax.axvspan(
    datetime.date(2020, 2, 19),
    datetime.date(2020, 4, 1),
    color='#AA3377',
    alpha=0.2,
    label='COVID-19 Pandemic',
)
ax.axvspan(
    datetime.date(2022, 1, 1),
    datetime.date(2022, 3, 12),
    color='#BBBBBB',
    alpha=0.4,
    label='Not Sure',
)
ax.plot(sp500_event.index, sp500_event['Close'])
ax.set_title('Huge crash after 2000s')
ax.set_xlabel('Year')
ax.set_ylabel('Closing Value')
ax.grid(axis='y')
plt.legend(loc='upper left', ncol=3)
plt.show()

## S&P500: Time interval for new all time high shorten<a id='sp500_new_high'></a>

In [ ]:
recent_20y_sp500 = sp500.loc['2000-01-01':].copy()
dataset_start = recent_20y_sp500.index[0].to_pydatetime()
dataset_end = recent_20y_sp500.index[-1].to_pydatetime()
days = abs(dataset_start - dataset_end).days
weeks = days // 7
recent_20y_sp500['New High'] = (
    recent_20y_sp500['Close'].asfreq('D').rolling(window=weeks * 7, min_periods=1).max()
)
new_high_occurs = recent_20y_sp500.drop_duplicates(subset='New High', keep='last')

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(recent_20y_sp500.index, recent_20y_sp500['Close'], lw=2)
ax.plot(recent_20y_sp500.index, recent_20y_sp500['New High'], lw=2)
[ax.axvline(x=d, color='r', alpha=0.2) for d in new_high_occurs.index]
custom_legends = [
    Line2D([0], [0], color='#0C5DA5', lw=2),
    Line2D([0], [0], color='#00B945', lw=2),
    Line2D([0], [0], color='r', lw=2),
]
ax.legend(
    custom_legends,
    ['Daily Close', 'New High Trend Line', 'New High Trigger'],
    loc='upper left',
    fontsize=20,
)
ax.set_title('Next New High', fontsize=30)
ax.grid(axis='y')
plt.show()

## S&P500: Most profitable month throughout the history<a id='sp500_month_bias'></a>

In [ ]:
profitable_month = sp500.copy()
profitable_month['Percent Change'] = profitable_month['Close'].pct_change() * 100
profitable_month['Month'] = pd.DatetimeIndex(
    profitable_month.reset_index()['Date']
).month

fig, ax = plt.subplots(1, 5, figsize=(15, 5))
plt.suptitle('Most profitable month in the history')
(profitable_month.groupby('Month').agg({'Percent Change': 'mean'}) * 100).plot(
    kind='bar', ax=ax[0], legend=False, title='Since 1927'
)
(
    profitable_month.loc['1950-01-01':].groupby('Month').agg({'Percent Change': 'mean'})
    * 100
).plot(kind='bar', ax=ax[1], legend=False, title='Since 1950')
(
    profitable_month.loc['1975-01-01':].groupby('Month').agg({'Percent Change': 'mean'})
    * 100
).plot(kind='bar', ax=ax[2], legend=False, title='Since 1975')
(
    profitable_month.loc['2000-01-01':].groupby('Month').agg({'Percent Change': 'mean'})
    * 100
).plot(kind='bar', ax=ax[3], legend=False, title='Since 2000')
(
    profitable_month.loc['2010-01-01':].groupby('Month').agg({'Percent Change': 'mean'})
    * 100
).plot(kind='bar', ax=ax[4], legend=False, title='Since 2010')
for i in range(0, 5):
    ax[i].yaxis.set_major_formatter(mtick.PercentFormatter())
    ax[i].grid(axis='y')
fig.tight_layout()
plt.show()

## S&P500 crossing 150 moving average == not a good timing<a id='sp500_good_timing'></a>

In [ ]:
recent_10y_sp500 = sp500.loc['2010-01-01':].copy()
recent_10y_sp500['150MA'] = recent_10y_sp500['Close'].rolling(150).mean()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(recent_10y_sp500.index, recent_10y_sp500['Close'], label='Closing Price')
ax.plot(recent_10y_sp500.index, recent_10y_sp500['150MA'], label='150 Moving Average')
ax.set_title('When to get in?')
ax.legend()
ax.grid(axis='y')
plt.show()

filled_sp500 = sp500.loc['1980-01-01':].asfreq('D').fillna(method='ffill')
filled_sp500['150MA'] = filled_sp500['Close'].rolling(150).mean()
below_150MA = filled_sp500[filled_sp500['150MA'] > filled_sp500['Close']].reset_index()
below_150MA['Consecutive Drop Group'] = (
    below_150MA['Date'].diff().dt.days.ne(1).cumsum()
)
below_150MA = below_150MA.groupby('Consecutive Drop Group').filter(
    lambda x: len(x) > 60
)
below_150MA_start = below_150MA.groupby('Consecutive Drop Group').first()
below_150MA_end = below_150MA.groupby('Consecutive Drop Group').last()

history_bad_duration = pd.concat(
    [below_150MA_start[['Date', 'Close']], below_150MA_end[['Date', 'Close']]], axis=1
)
history_bad_duration.columns = ['Start Date', 'Start Close', 'End Date', 'End Close']
history_bad_duration.reset_index(drop=True, inplace=True)
history_bad_duration['Day Interval'] = (
    history_bad_duration['End Date'] - history_bad_duration['Start Date']
).dt.days
history_bad_duration['Close Interval'] = (
    history_bad_duration['End Close'] - history_bad_duration['Start Close']
)
history_bad_duration = history_bad_duration[
    [
        'Start Date',
        'End Date',
        'Day Interval',
        'Start Close',
        'End Close',
        'Close Interval',
    ]
]
display(history_bad_duration)

## Getting into the fundamental stuff<a id='fundamental_stuff'></a>

In [ ]:
def ticker_financials(ticker: str):
    input_ticker = copy.copy(ticker.upper())
    url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(
        input_ticker, input_ticker
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=5)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    start = script_data.find('context') - 2
    json_data = json.loads(script_data[start:-12])

    annual_income_statement = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['incomeStatementHistory']['incomeStatementHistory']
    quarterly_income_statement = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['incomeStatementHistoryQuarterly']['incomeStatementHistory']
    annual_cashflow_statement = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['cashflowStatementHistory']['cashflowStatements']
    quarterly_cashflow_statement = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['cashflowStatementHistoryQuarterly']['cashflowStatements']
    annual_balance_sheet = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['balanceSheetHistory']['balanceSheetStatements']
    quarterly_balance_sheet = json_data['context']['dispatcher']['stores'][
        'QuoteSummaryStore'
    ]['balanceSheetHistoryQuarterly']['balanceSheetStatements']

    (
        a_income_statement,
        q_income_statement,
        a_cashflow_statement,
        q_cashflow_statement,
        a_balance_sheet,
        q_balance_sheet,
    ) = ([] for i in range(6))

    for s1, s2, s3, s4, s5, s6 in zip(
        annual_income_statement,
        quarterly_income_statement,
        annual_cashflow_statement,
        quarterly_cashflow_statement,
        annual_balance_sheet,
        quarterly_balance_sheet,
    ):

        statement1, statement2, statement3, statement4, statement5, statement6 = (
            {} for i in range(6)
        )

        for (k1, v1), (k2, v2), (k3, v3), (k4, v4), (k5, v5), (k6, v6) in zip(
            s1.items(), s2.items(), s3.items(), s4.items(), s5.items(), s6.items()
        ):
            try:
                statement1[k1] = v1['longFmt']
                statement2[k2] = v2['longFmt']
                statement3[k3] = v3['longFmt']
                statement4[k4] = v4['longFmt']
                statement5[k5] = v5['longFmt']
                statement6[k6] = v6['longFmt']

            except TypeError:
                continue

            except KeyError:
                continue

        a_income_statement.append(statement1)
        q_income_statement.append(statement2)
        a_cashflow_statement.append(statement3)
        q_cashflow_statement.append(statement4)
        a_balance_sheet.append(statement5)
        q_balance_sheet.append(statement6)

    a_income_statement = pd.DataFrame(a_income_statement)
    q_income_statement = pd.DataFrame(q_income_statement)
    a_cashflow_statement = pd.DataFrame(a_cashflow_statement)
    q_cashflow_statement = pd.DataFrame(q_cashflow_statement)
    a_balance_sheet = pd.DataFrame(a_balance_sheet)
    q_balance_sheet = pd.DataFrame(q_balance_sheet)

    return (
        a_income_statement,
        q_income_statement,
        a_cashflow_statement,
        q_cashflow_statement,
        a_balance_sheet,
        q_balance_sheet,
    )

In [ ]:
def ticker_everything(ticker: str):
    ticker = yf.Ticker(ticker)
    ticker_info = (
        pd.Series(ticker.info)
        .to_frame()
        .rename(columns={0: 'Value'})
        .rename_axis('Info')
    )
    ticker_actions = ticker.actions
    ticker_afinancials = ticker.financials
    ticker_qfinancials = ticker.quarterly_financials
    ticker_abalancesheet = ticker.balancesheet
    ticker_qbalancesheet = ticker.quarterly_balancesheet
    ticker_cashflowstatement = ticker.cashflow
    ticker_qcashflowstatement = ticker.quarterly_cashflow
    ticker_earnings = ticker.earnings
    ticker_sus = ticker.sustainability
    ticker_recom = ticker.recommendations
    ticker_cal = ticker.calendar

    return (
        ticker_info,
        ticker_actions,
        ticker_afinancials,
        ticker_qfinancials,
        ticker_abalancesheet,
        ticker_qbalancesheet,
        ticker_cashflowstatement,
        ticker_qcashflowstatement,
        ticker_earnings,
        ticker_sus,
        ticker_recom,
        ticker_cal,
    )

In [ ]:
def ticker_statistics(ticker: str):
    input_ticker = copy.copy(ticker.upper())
    url = "https://finance.yahoo.com/quote/{}/key-statistics?p={}".format(
        input_ticker, input_ticker
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=5)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    start = script_data.find('context') - 2
    json_data = json.loads(script_data[start:-12])
    ticker_stats = json_data['context']['dispatcher']['stores']['QuoteSummaryStore'][
        'defaultKeyStatistics'
    ]
    display(ticker_stats)
    ticker_stats = pd.DataFrame(ticker_stats).T

    return ticker_stats

## Intrinsic Value <a id='intrinsic_value'></a>

To purchase a stock at the 'right place', intrinsic value or underlying value is one of the metrics to estimate does the current price of specific stock is underweight or overvalued based on the fundamental metrics of the company. Abundance formula has been formed to calculate the intrinsic value, here a simple formula to calculate the intrinsic value. The formula requires following value to compute the intrinsic value of the business which encompassing:

|      Params     |                  Name                 |   Unit   | Timeframe |         From        |
|:---------------:|:-------------------------------------:|:--------:|:---------:|:-------------------:|
|      T_OCF      |          Operating Cash Flow          | Millions |  Last 4Q  | Cash Flow Statement |
|   T_Total_Debt  |               Total Debt              | Millions |   Last Q  |    Balance Sheet    |
|      T_CSTI     |      Cash & Short Term Investment     | Millions |   Last Q  |    Balance Sheet    |
|     T_CFGR1     |  Cash flow Growth rate next 1-5 years |  Percent |    ANY    |     EPS Estimate    |
|     T_CFGR2     | Cash flow Growth rate next 6-10 years |  Percent |    ANY    |     EPS Estimate    |
|      T_NSO      |         No. Shares Outstanding        |    No.   |   Recent  |          -          |
|   T_Last_Close  |               Last Close              |   Price  | Yesterday |          -          |
| T_Discount_Rate |             Discount Rate             |  Percent |    ANY    |         Beta        |
|  T_next_n_year  |            Projected Years            |   Year   |    ANY    |         Self        |

Beta -> Discount Rate Conversion <br>
\<.8 = 4.6% <br>
1 ~ 1.5 = 5.6% ~ 8.1% | +0.5% per 0.1Beta <br>
\> 1.6 = 8.6% <br>

In [ ]:
def intrinsic_value(
    T_OCF,
    T_Total_Debt,
    T_CSTI,
    T_CFGR1,
    T_CFGR2,
    T_NSO,
    T_Last_Close,
    T_Discount_Rate,
    T_next_n_year,
):
    OCF = T_OCF
    Total_Debt = T_Total_Debt
    CSTI = T_CSTI
    CFGR1 = T_CFGR1 / 100
    CFGR2 = T_CFGR2 / 100
    NSO = T_NSO
    Last_Close = T_Last_Close
    Discount_Rate = T_Discount_Rate / 100

    next_n_year = T_next_n_year
    POCF = list()
    POCF.append(OCF * (1 + CFGR1))
    Discount_Factor = list()
    Discount_Factor.append(1 / (1 + Discount_Rate))
    Discount_Value = list()
    Discount_Value.append(POCF[-1] * Discount_Factor[-1])

    for i in range(next_n_year - 1):
        POCF.append(POCF[-1] * (1 + CFGR1))
        Discount_Factor.append(Discount_Factor[-1] * (1 / (1 + Discount_Rate)))
        Discount_Value.append(POCF[-1] * Discount_Factor[-1])

    total_n_year_CF = sum(Discount_Value)

    INTRINSIC_VALUE_before_CashDebt = total_n_year_CF / NSO
    Debt_per_share = Total_Debt / NSO
    Cash_per_share = CSTI / NSO

    INTRINSIC_VALUE = INTRINSIC_VALUE_before_CashDebt - Debt_per_share + Cash_per_share
    Discount_OR_Premium = ((Last_Close - INTRINSIC_VALUE) / INTRINSIC_VALUE) * 100

    return round(INTRINSIC_VALUE, 2), round(Discount_OR_Premium, 2)

In [ ]:
def compute_intrinsic_value(ticker: str, display=False):
    input_ticker = copy.copy(ticker.upper())
    url = "https://finance.yahoo.com/quote/{}/key-statistics?p={}".format(
        input_ticker, input_ticker
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=5)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    start = script_data.find('context') - 2
    json_data = json.loads(script_data[start:-12])
    ticker_stats = json_data['context']['dispatcher']['stores']['QuoteSummaryStore'][
        'defaultKeyStatistics'
    ]
    ticker_stats = pd.DataFrame(ticker_stats).T
    ticker = yf.Ticker(ticker)
    ticker_info = (
        pd.Series(ticker.info)
        .to_frame()
        .rename(columns={0: 'Value'})
        .rename_axis('Info')
    )
    ticker_qcashflowstatement = ticker.quarterly_cashflow
    ticker_qbalancesheet = ticker.quarterly_balancesheet
    OCF = sum(
        ticker_qcashflowstatement.loc['Total Cash From Operating Activities'].values
    )
    try:
        TDEBT = ticker_qbalancesheet.loc['Long Term Debt'].values[0]
    except:
        TDEBT = ticker_info.loc['totalDebt'].values[0]
    CSTI = (
        ticker_qbalancesheet.loc['Cash'].values[0]
        + ticker_qbalancesheet.loc['Short Term Investments'].values[0]
    )
    CFGR1 = 15
    CFGR2 = 5
    NSO = ticker_stats.loc['sharesOutstanding'].values[0]
    history = ticker.history()
    LASTCLOSE = ticker.history().tail(1)['Close'].values[0]
    BETA = 1.1
    YEAR_TO_PROJECT = 10

    INTRINSIC_VALUE, Discount_OR_Premium = intrinsic_value(
        T_OCF=OCF,
        T_Total_Debt=TDEBT,
        T_CSTI=CSTI,
        T_CFGR1=CFGR1,
        T_CFGR2=CFGR2,
        T_NSO=NSO,
        T_Last_Close=LASTCLOSE,
        T_Discount_Rate=BETA,
        T_next_n_year=YEAR_TO_PROJECT,
    )

    if display:
        print('Input Ticker: {}'.format(input_ticker))
        print('Current Price: {}'.format(round(LASTCLOSE, 2)))
        print('Instrinsic Value: {}'.format(INTRINSIC_VALUE))
        print('(Discount)/Premium: {}%'.format(Discount_OR_Premium))
        print('\n')

    return INTRINSIC_VALUE, round(LASTCLOSE, 2)

In [ ]:
def get_everything(input_ticker):
    a_is, q_is, a_cs, q_cs, a_bs, q_bs = ticker_financials(ticker=input_ticker)
    display(a_is, q_is, a_cs, q_cs, a_bs, q_bs)

    (
        ticker_info,
        ticker_actions,
        ticker_afinancials,
        ticker_qfinancials,
        ticker_abalancesheet,
        ticker_qbalancesheet,
        ticker_cashflowstatement,
        ticker_qcashflowstatement,
        ticker_earnings,
        ticker_sus,
        ticker_recom,
        ticker_cal,
    ) = ticker_everything(ticker=input_ticker)
    display(
        ticker_info,
        ticker_actions,
        ticker_afinancials,
        ticker_qfinancials,
        ticker_abalancesheet,
        ticker_qbalancesheet,
        ticker_cashflowstatement,
        ticker_qcashflowstatement,
        ticker_earnings,
        ticker_sus,
        ticker_recom,
        ticker_cal,
    )

    ticker_stats = ticker_statistics(ticker=input_ticker)
    display(ticker_stats)

    compute_intrinsic_value(input_ticker)

In [ ]:
raw_data = list()
for t in tqdm.tqdm(
    ['msft', 'googl', 'nvda', 'aapl', 'fb', 'tsla', 'amzn', 'crm', 'amd', 'adbe']
):
    itsv, lc = compute_intrinsic_value(t)
    mkc = yf.Ticker(t).info['marketCap'] / 1000000000
    raw_data.append([itsv, lc, mkc, t.upper()])

In [ ]:
data = pd.DataFrame(raw_data, columns=['itsv', 'lc', 'mkc', 'tksb'])
data_json = data.to_json(orient='records')
data_json = json.loads(data_json)
data

## Undervalue Companies huh <a id='undervalue_companies'></a>

In [ ]:
%%javascript

    var script = document.createElement('script');
    script.type = 'text/javascript';
    script.src = "https://d3js.org/d3.v7.min.js";
    document.head.appendChild(script);
    console.log(window.d3)

In [ ]:
svg_script = '''

const data = %s

console.log(data)

const margin = {top: 80, right: 30, bottom: 50, left: 60};
const width = 800 - margin.left - margin.right;
const height = 400 - margin.top - margin.bottom;

const svg = d3.select(element)
    .append('svg')
    .attr('width', width + margin.left + margin.right)
    .attr('height', height + margin.top + margin.bottom)
    .style('background-color', '#f2e8dc');
    
const main = svg.append('g')
    .attr('class', 'focus')
    .attr('transform', `translate(${margin.left},${margin.top})`);

// Add xaxis
const x = d3.scaleLinear()
    .domain([0, d3.max(data, d => d.lc)])
    .range([0, width]);
main.append('g')
    .attr('transform', `translate(0, ${height})`)
    .call(d3.axisBottom(x));
main.append('text')
    .attr('x', width * 0.90)
    .attr('y', height + 40)
    .attr('font-size', '13px')
    .attr('font-weight', 'bold')
    .attr('font-family', 'sans-serif')
    .text('Last Close →')

// Add yaxis
const y = d3.scaleLinear()
    .domain([0, d3.max(data, d => d.itsv)])
    .range([height, 0]);
main.append('g')
    .call(d3.axisLeft(y))
main.append('text')
    .attr('x', -(width) * 0.07)
    .attr('y', -20)
    .attr('font-size', '13px')
    .attr('font-weight', 'bold')
    .attr('font-family', 'sans-serif')
    .text('↑ Intrinsic Value')

// Add grid
const grid = g => g
    .attr('stroke', 'black')
    .attr('stroke-opacity', 0.2)
    .call(g => g.append('g')
      .selectAll('line')
      .data(x.ticks())
      .join('line')
        .attr('x1', d => 0 + x(d))
        .attr('x2', d => 0 + x(d))
        .attr('y1', 0)
        .attr('y2', height))
    .call(g => g.append('g')
        .selectAll('line')
        .data(y.ticks())
        .join('line')
          .attr('y1', d => 0 + y(d))
          .attr('y2', d => 0 + y(d))
          .attr('x1', 0)
          .attr('x2', width));
main.append('g')
    .call(grid);
    
const dotScale = d3.scaleSqrt()
    .domain(d3.extent(data, d => d.mkc))
    .range([5, 20]);
    
const dotTooltip = d3.select('body').append('div')
    .attr('class', 'dotTooltip')
    .style('opacity', 0);
    
const dotColor = d3.scaleOrdinal()
    .domain(data.map(d => d.tksb))
    .range(d3.schemeCategory10);
    
// Add dot
main.append('g')
    .selectAll('dot')
    .data(data)
    .enter()
    .append('circle')
    .attr('fill', 'gray')
    .attr('stroke', 'none')
    .attr('cx', d => x(d.lc))
    .attr('cy', d => y(d.itsv))
    .attr('r', d => dotScale(d.mkc))
    .attr('fill', d => dotColor(d.tksb))
    .on('mouseover', function(event, d){
        d3.select(this).attr('r', d => dotScale(d.mkc) * 1.5);
        d3.select(this).attr('stroke', '#333').attr('stroke-width', 2);
        svg.append('text')
        .attr('class', 'dot-text')
        .attr('fill', dotColor(d.tksb))
        .attr('font-size', '16px')      
        .attr('text-anchor', 'middle')
        .attr('font-weight', 'bold')
        .attr('x', (width + margin.left) / 2 ) 
        .attr('y', (height) * 0.25 )
        .text(d.tksb);
    })
    .on('mouseout', function(d){
        d3.select(this).attr('r', d => dotScale(d.mkc));
        d3.select(this).attr('stroke', null);
        svg.selectAll('.dot-text').remove();

    });
    
// Add title
const title = main.append('g')
        .attr('class', 'title')
        .attr('transform', 'translate(20,20)')
        .style('font-size', '25px')
        .style('text-anchor', 'middle');
title.append('text')
    .style('font-weight', 'bold')
    .attr('x', width*0.73)
    .attr('y', -70)
    .text('Finding Undervalue Companies');
title.append('text')
    .style('font-weight', 'bold')
    .style('fill', '#2F3030')
    .style('font-size', '12px')
    .attr('x', width*0.79)
    .attr('y', -50)
    .text('Think of a companies as money making machine')
    
// Add dot scale legend
const dotLegendValues = [500, 1000, 2000]
const dotLegendX = width * 0.12;
const dotLegendY = height * 0.10;
const dotScaleLegend = svg.append('g')
    .attr('class', 'dotScaleLegend')
    .attr('transform', `translate(${dotLegendX}, ${dotLegendY})`);
dotScaleLegend.append('g')
    .selectAll('dot')
    .data(dotLegendValues)
    .enter()
    .append('circle')
    .attr('cx', dotLegendX)
    .attr('cy', function(d){ return dotLegendY - dotScale(d) } )
    .attr('r', function(d){ return dotScale(d) })
    .attr('stroke', 'black')
    .style('fill', 'none')
dotScaleLegend.append('text')
    .attr('x', dotLegendX - 32)
    .attr('y', dotLegendY + 15)
    .attr('font-weight', 'bold')
    .style('font-size', '12px')
    .text('Market Cap')
    
// Add dot color legend
const dotColorLegendY = -(height) * 0.24
const dotColorlegend = main.append('g')
    .attr('class', 'dotColorlegend')
    .attr('transform', 'translate(20,20)');
dotColorlegend.append('rect')
    .attr('width', 10)
    .attr('height', 10)
    .attr('fill', d3.schemeCategory10[0])
    .attr('stroke', d3.schemeCategory10[0])
    .attr('stroke-width', 1.5)
    .attr('x', width * 0.25)
    .attr('y', dotColorLegendY);
dotColorlegend.append('rect')
    .attr('width', 10)
    .attr('height', 10)
    .attr('fill', d3.schemeCategory10[1])
    .attr('stroke', d3.schemeCategory10[1])
    .attr('stroke-width', 1.5)
    .attr('x', width * 0.27)
    .attr('y', dotColorLegendY);
dotColorlegend.append('rect')
    .attr('width', 10)
    .attr('height', 10)
    .attr('fill', d3.schemeCategory10[2])
    .attr('stroke', d3.schemeCategory10[2])
    .attr('stroke-width', 1.5)
    .attr('x', width * 0.29)
    .attr('y', dotColorLegendY);
dotColorlegend.append('text')
    .attr('x', width * 0.22)
    .attr('y', -(height) * 0.12)
    .text('Ticker Symbol')
    .attr('font-weight', 'bold')
    .style('font-size', '12px')
    
// Add footnotes
const footnotes = main.append('g')
    .attr('class', 'footnotes')
footnotes.append('text')
     .attr('x', -(width) * 0.06)
     .attr('y', (height + margin.bottom) * 0.95)
     .attr('font-size', '12px')
     .text('Source: Calculated by Lee Kiong Wei Jie based on Yahoo Finance.') 

''' % json.dumps(
    data_json
)
Javascript(svg_script)

# Dependencies<a id='dependencies'></a>

In [ ]:
# !pip install yfinance==0.1.63
# !pip install pandas==1.3.5
# !pip install numpy==1.21.4
# !pip install matplotlib==3.5.1
# !pip install scipy==1.5.4
# !pip install seaborn==0.11.1
# !pip install sklearn==0.23.2
# !pip install requests==2.27.1
# !pip install beautifulsoup4==4.9.1
# !pip install pandas_datareader==0.9.0
# !pip install tqdm==4.62.3